In [42]:
import os
from fastapi import FastAPI, Request
import json
from utility.DBConInfo import dbConInfo
from utility.Env import env

ImportError: cannot import name 'ParamSpec'

In [40]:
app = FastAPI()
#CORS(app)

@app.post("/available_table")
def available_table():
    try:
        dci=dbConInfo(env.host(),env.user(),env.password(),env.dbname(),env.sslmode())
        df=dci.readSQL("SELECT * FROM pg_catalog.pg_tables")
        df_public=df[df.schemaname=="public"]
        list_public=df_public.tablename.values.tolist()
        return json.dumps({"status":"success","result":list_public})
    
    except Exception as e:
        return json.dumps({"status":"error","result":str(e)})
        
@app.post("/query_table")
def query_table(request: Request):
    try:
        dci=dbConInfo(env.host(),env.user(),env.password(),env.dbname(),env.sslmode())
        index = Request.form.get('index')
        column = Request.form.get('column') if Request.form.get('column') else '*'
        active = "where active='Y'" if request.form.get('active')=="Y" else ''
        sql=f"SELECT {column} FROM public.{index} {active}"
        df=dci.readSQL(sql)
        df_header = df.columns.values.tolist()
        df_list = df.values.tolist()
        df_list.insert(0, df_header)
        return json.dumps({"status":"success","result":df_list})

    except Exception as e:
        return json.dumps({"status":"error","result":str(e)})
        

In [41]:
if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


192.168.153.167 - - [16/Jun/2022 13:22:31] "POST /query_table HTTP/1.1" 200 -


aps_md
